## Check Mediastack Dataset

#### Metadatata

In [1]:
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

In [2]:
import sys
sys.path.append('../utils')
import functions
import source_country_mapping

In [ ]:
mediastack_df = pd.read_csv('../data/raw/news_sentiment_analysis.csv')

mediastack_df.head()

#### Dataset overview

In [ ]:
functions.show_basic_info(mediastack_df)

In [ ]:
functions.show_data_types(mediastack_df)

In [ ]:
functions.show_missing_values(mediastack_df)

In [ ]:
functions.check_for_duplicates(mediastack_df)

#### Check earliest and latest date in the dataset

In [ ]:
earliest_date = mediastack_df['Published At'].min()
latest_date = mediastack_df['Published At'].max()

print(f'Earliest date in the dataset: {earliest_date}')
print(f'Latest date in the dataset: {latest_date}')

### **Data Cleaning**

1. Standardize column names
2. Replace empty values in the 'author' column with 'Unknown'
3. Standardize publishing date format
4. Convert columns to lowercase
5. Clean source names
6. Remove non-English rows


1. Rename and reorder columns

In [9]:
rename_dict = {
    'Source': 'source',
    'Author': 'author',
    'Title': 'title',
    'Description': 'description',
    'URL': 'url',
    'Published At': 'published_at',
    'Sentiment': 'sentiment',
    'Type': 'category'
}

mediastack_df = functions.rename_columns(mediastack_df, rename_dict)

In [10]:
new_order = ['author', 'title', 'description', 'url', 'source', 'category', 'published_at', 'sentiment']
mediastack_df = mediastack_df[new_order]

2. Replace empty values in the 'author' column with 'Unknown'

In [11]:
def replace_empty_values(df, column_name, default_value):
    """
    Replaces empty or missing values in the specified column with a default value.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame to modify.
    - column_name (str): The name of the column to check for empty values.
    - default_value (str): The value to replace empty or missing values with.
    
    Returns:
    - pd.DataFrame: The modified DataFrame with empty values replaced.
    """
    df[column_name] = df[column_name].replace(['', None], default_value)
    return df

In [12]:
mediastack_df = replace_empty_values(mediastack_df, 'author', 'Unknown')

3. Standardize publishing date format

In [13]:
def standardize_date_format(df, column_name):
    """
    Standardizes the date format in the specified column to 'YYYY-MM-DD'.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame to modify.
    - column_name (str): The name of the column containing the date to standardize.
    
    Returns:
    - pd.DataFrame: The modified DataFrame with the standardized date format.
    """
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce').dt.strftime('%Y-%m-%d')
    return df

In [14]:
mediastack_df = standardize_date_format(mediastack_df, 'published_at')
# mediastack_df.head()

4. Columns to lowercase

In [15]:
columns_to_convert = ['category', 'source']

for column in columns_to_convert:
    functions.convert_strings_to_lowercase(mediastack_df, column)

5. Clean source names

In [16]:
mediastack_df['source'] = mediastack_df['source'].apply(functions.clean_source_names)
# mediastack_df.head()

6. Remove non-English rows

In [17]:
DetectorFactory.seed = 0    # seed for consistency in language detection

def detect_languages_and_filter(df, text_column):
    """
    Detects languages in a specified column of a Pandas DataFrame and filters out non-English rows.

    Args:
        df (pd.DataFrame): DataFrame containing the text data.
        text_column (str): The name of the column with the text data.

    Returns:
        pd.DataFrame: Filtered DataFrame containing only English rows.
    """
    detected_languages = []

    for text in df[text_column]:
        try:
            lang = detect(text)
            detected_languages.append(lang)
        except LangDetectException:
            detected_languages.append('unknown')

    df['detected_language'] = detected_languages

    filtered_df = df[df['detected_language'] == 'en']   # filter out rows in English

    filtered_df = filtered_df.drop(columns=['detected_language'])   # drop the 'detected_language' column

    return filtered_df

filtered_df = detect_languages_and_filter(mediastack_df, 'description')

In [ ]:
display(filtered_df)

#### Value counts

In [ ]:
print('Value counts for category column:')
functions.show_column_value_counts(filtered_df, 'category')
print('Value counts for clean_source column:')
functions.show_column_values(filtered_df, 'clean_source')

In [ ]:
functions.show_column_values(filtered_df, 'source')

#### Check Filtered DF for missing values and duplicates

In [ ]:
functions.show_missing_values(filtered_df)

In [ ]:
functions.check_for_duplicates(filtered_df)

In [ ]:
filtered_df = functions.remove_duplicates(filtered_df)

#### Clean 'description' column

In [ ]:
# show the full text in the columns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50) 

print(filtered_df['description'].head(10))

#### Check description and title columns

In [25]:
filtered_df['clean_description'] = filtered_df['description'].apply(functions.clean_text)
filtered_df['clean_title'] = filtered_df['title'].apply(functions.clean_text)

In [ ]:
print(filtered_df['clean_description'].head(10))

In [ ]:
print(filtered_df['clean_title'].head(20))

### Add country columns (name and code) to the dataframe using source country dictionary

In [28]:
filtered_df = source_country_mapping.add_country_column(filtered_df, 'source')

In [29]:
filtered_df = source_country_mapping.add_country_code_column(filtered_df, 'country')

In [ ]:
filtered_df.head()

#### Create a new CSV file with the cleaned data

In [32]:
# filtered_df.to_csv('../data/clean/1_mediastack_news_cleaned.csv', index=False)